In [1]:
import numpy as np
from sklearn.cluster import KMeans


In [2]:
def readArchive(fileName):
    with open(fileName, 'r') as file:
        lines = file.readlines()

    # Elimina los caracteres de salto de línea y divide los valores por comas
    data = [line.strip().split(',') for line in lines]

    # Convierte la lista en una matriz de numpy
    return np.array(data, dtype=int)

dataTrain = readArchive('optdigits.tra')

X_train = dataTrain[:, :64]
Y_train = dataTrain[:, 64]

dataTest = readArchive('optdigits.tes')
X_test = dataTest[:, :64]
Y_test = dataTest[: , 64]

In [3]:

def initialize_centroids_with_features(images, labels, num_clusters):
    # Preprocesamiento de las imágenes
    flattened_images = images.reshape(images.shape[0], -1)  # Aplanar las imágenes en vectores de características
    unique_labels = np.unique(labels)

    # Cálculo de características promedio por etiqueta
    centroids = []
    for label in unique_labels:
        label_images = flattened_images[labels == label]
        label_mean = np.mean(label_images, axis=0)
        centroids.append(label_mean)

    # Selección de los primeros K centroides iniciales
    centroids = np.array(centroids)[:num_clusters]

    return centroids

# Ejemplo de uso
# Supongamos que tienes un conjunto de datos de imágenes 'images' de dimensiones (n, 8, 8) y sus etiquetas correspondientes 'labels'
# donde 'n' es el número de imágenes.

# Especifica el número de clústeres deseado
num_clusters = 10

# Inicialización de centroides con características
initial_centroids = initialize_centroids_with_features(X_test, Y_test, num_clusters)
# Preprocesamiento de las imágenes
flattened_images = X_test.reshape(X_test.shape[0], -1)  # Aplanar las imágenes en vectores de características

# Ejecución del algoritmo K-means
kmeans = KMeans(n_clusters=num_clusters, init=initial_centroids, n_init=1)
kmeans.fit(flattened_images)

# Obteniendo los centroides finales
final_centroids = kmeans.cluster_centers_


In [4]:
# Supongamos que tienes las etiquetas originales de las imágenes en la variable 'true_labels'
true_labels = Y_test

# Obtener las etiquetas asignadas por K-means
predicted_labels = kmeans.labels_

# Calcular la precisión
accuracy = np.mean(true_labels == predicted_labels) * 100

# Imprimir la precisión
print("Precisión de clasificación: {:.2f}%".format(accuracy))


Precisión de clasificación: 86.31%


In [5]:
import numpy as np

def initialize_centroids(data, k):
    # Inicialización de centroides de manera aleatoria
    np.random.seed(0)
    indices = np.random.choice(data.shape[0], k, replace=False)
    centroids = data[indices]
    return centroids

def assign_clusters(data, centroids):
    # Asignación de puntos a clústeres según la distancia euclidiana
    distances = np.sqrt(((data[:, np.newaxis] - centroids) ** 2).sum(axis=2))
    labels = np.argmin(distances, axis=1)
    return labels

def update_centroids(data, labels, k):
    # Actualización de los centroides como la media de los puntos asignados a cada clúster
    centroids = np.zeros((k, data.shape[1]))
    for i in range(k):
        cluster_data = data[labels == i]
        if len(cluster_data) > 0:
            centroids[i] = np.mean(cluster_data, axis=0)
    return centroids

def kmeans(data, k, max_iterations=10000):
    centroids = initialize_centroids_with_features(data, Y_train, k)

    for _ in range(max_iterations):
        prev_centroids = centroids.copy()

        labels = assign_clusters(data, centroids)
        centroids = update_centroids(data, labels, k)

        # Comprobar convergencia
        if np.all(prev_centroids == centroids):
            break

    return labels, centroids

# Ejemplo de uso
# Supongamos que tienes un conjunto de datos 'data' de dimensiones (n, d)
# donde 'n' es el número de puntos y 'd' es el número de dimensiones.

# Especifica el número de clústeres deseado
k = 10

# Ejecución del algoritmo K-means
labels, final_centroids = kmeans(X_train, k)

# Imprimir los centroides finales
print("Centroides finales:")
print(final_centroids)

# Imprimir las etiquetas asignadas a cada punto
print("Etiquetas asignadas:")
print(labels)


Centroides finales:
[[0.00000000e+00 2.96495957e-02 4.60377358e+00 1.32264151e+01
  1.09433962e+01 2.61994609e+00 5.92991914e-02 0.00000000e+00
  0.00000000e+00 1.18328841e+00 1.30404313e+01 1.30700809e+01
  1.22425876e+01 1.07331536e+01 7.33153639e-01 0.00000000e+00
  0.00000000e+00 3.90835580e+00 1.44204852e+01 4.59299191e+00
  2.95417790e+00 1.27331536e+01 3.31805930e+00 0.00000000e+00
  0.00000000e+00 5.25876011e+00 1.27789757e+01 1.56873315e+00
  2.66846361e-01 9.18598383e+00 6.64150943e+00 0.00000000e+00
  0.00000000e+00 5.46900270e+00 1.19272237e+01 8.65229111e-01
  1.05121294e-01 8.64690027e+00 7.38814016e+00 0.00000000e+00
  0.00000000e+00 3.35849057e+00 1.35579515e+01 1.68194070e+00
  1.38544474e+00 1.13800539e+01 6.46361186e+00 0.00000000e+00
  0.00000000e+00 7.73584906e-01 1.32183288e+01 1.05202156e+01
  1.05983827e+01 1.34824798e+01 2.67654987e+00 0.00000000e+00
  0.00000000e+00 1.88679245e-02 4.61725067e+00 1.36064690e+01
  1.33261456e+01 5.54986523e+00 1.85983827e-01 0.0

In [6]:
true_labels = Y_test

def calculate_confusion_matrix(true_labels, predicted_labels, num_classes):
    confusion_matrix = np.zeros((num_classes, num_classes))
    for true_label, predicted_label in zip(true_labels, predicted_labels):
        confusion_matrix[true_label][predicted_label] += 1
    return confusion_matrix
# Calcula la matriz de confusión
confusion_matrix = calculate_confusion_matrix(Y_train, labels, num_classes=k)

# Imprime la matriz de confusión
print("Matriz de Confusión:")
print(confusion_matrix)

# Calcula la precisión como la suma de las coincidencias diagonales dividida por el número total de puntos
accuracy = np.sum(np.max(confusion_matrix, axis=1)) / len(X_train)

# Imprime la precisión
print("Precisión de clasificación: {:.2f}%".format(accuracy * 100))

Matriz de Confusión:
[[371.   1.   0.   0.   2.   0.   1.   0.   0.   1.]
 [  0. 327.  17.   0.   0.   3.   4.   2.  11.  25.]
 [  0.   0. 335.   3.   0.   0.   1.   7.  24.  10.]
 [  0.   2.   3. 352.   0.   2.   0.   8.   7.  15.]
 [  0.   7.   0.   0. 329.   7.   5.  30.   5.   4.]
 [  0.   1.   0.   3.   0. 267.   2.   0.   0. 103.]
 [  0.   3.   0.   0.   1.   0. 373.   0.   0.   0.]
 [  0.   3.   0.   1.   0.   0.   0. 378.   5.   0.]
 [  0.  20.   0.   5.   2.   3.   4.   2. 337.   7.]
 [  0.   5.   0.   3.   6.   7.   0.  33.   2. 326.]]
Precisión de clasificación: 88.80%


In [7]:
from sklearn.metrics import adjusted_rand_score

# Supongamos que tienes las etiquetas verdaderas de clase en la variable 'true_labels'
true_labels = Y_train

# Calcula el ARI entre las etiquetas verdaderas y las asignaciones de clústeres
ari = adjusted_rand_score(true_labels, labels)

# Imprime el ARI
print("Índice de Rand Ajustado (ARI): {:.4f}".format(ari))


Índice de Rand Ajustado (ARI): 0.7743
